In [ ]:
# 날짜, 증권사, 제목을 가진 pdf파일 생성
# 각 파일을 text로 변환

In [ ]:
pip install pdfminer.six

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import urllib.request

In [ ]:
BASE_DIR = os.getcwd()

In [ ]:
REPORTS_DIR = BASE_DIR

In [ ]:
url = 'https://finance.naver.com/research/debenture_list.nhn?&page={}'.format('1')
response = requests.get(url)
# response
# response.url
# response.content

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')
# soup

In [ ]:
reports = soup.find('table', class_='type_1').find_all('tr')[1:]
reports
len(reports)

In [ ]:
pdf_list = []
# for문 준비 
for td in reports:
    filename = ""
    
    # date 코드
    date = td.find('td', class_='date')
    if date is not None:
        date = date.text
        # print(date)
        filename += date
        filename += '_'
        # print(filename)
        
    # author 코드
    author = td.find_all('td')
    if len(author) == 5:
        author = author[1].text
#       print(len(author))
#       print(author)
        filename += author
        filename += '_'
        # print(filename)
    
    # title 코드 
    title = td.find('td').find('a')
    if title is not None:
        title = title.text
        filename += title
        filename = filename.replace('?','')
        pdf_list.append(filename + '.pdf')
        # print(title)
        # print(filename)

    # pdf 코드
    pdf_href = td.find('td', class_='file')
    if pdf_href is not None:
        pdf_href = str(pdf_href.find('a')['href'])
        # print(pdf_href)
        # print(type(pdf_href))


        pdf_url = pdf_href
        print(pdf_url)
        # URL of the image to be downloaded is defined as image_url 
        r = requests.get(pdf_url) # create HTTP response object 

        # send a HTTP request to the server and save 
        # the HTTP response in a response object called r 
        naver_reports_path = os.path.join(REPORTS_DIR, filename + '.pdf') 
        # with open(filename + '.pdf','wb') as f:
        with open(naver_reports_path, 'wb') as f:
            # write the contents of the response (r.content) 
            # to a new file in binary mode. 
            f.write(r.content)

In [ ]:
pdf_list

In [ ]:
from PyPDF2 import PdfFileReader

def getTextPDF(pdfFileName, password = ''):
    pdf_file = open(pdfFileName, 'rb')
    read_pdf = PdfFileReader(pdf_file)
    
    if password != '':
        read_pdf.decrypt(password)
        
    text = []
    for i in range(0, read_pdf.getNumPages()):
        text.append(read_pdf.getPage(i).extractText())
        
    return '\n'.join(text)

In [ ]:
import sys
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
import io

BASE_DIR = os.getcwd()
REPORTS_DIR = BASE_DIR

def pdfparser(data):
    try:
        fp = open(data, 'rb')
        rsrcmgr = PDFResourceManager()
        retstr = io.StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
        # Create a PDF interpreter object.
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        # Process each page contained in the document.

        for page in PDFPage.get_pages(fp):
            interpreter.process_page(page)
            data =  retstr.getvalue()
            text_path = os.path.join(REPORTS_DIR, i + '.txt')
            with open(text_path, 'w', encoding='utf-8') as f:
                f.write(data)
                
    except Attribute Error:
        pass
    
if __name__ == '__main__':
    for i in pdf_list:
        pdfparser(i)